In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import helper_functions as hf
from IPython.display import clear_output
import os
import mysql.connector
from config import conn_host, conn_database, conn_user, conn_password
import json

In [2]:
def connect_to_db():
    return mysql.connector.connect(host=conn_host, 
                                     database=conn_database,
                                     user=conn_user,
                                     password=conn_password)

def execute_query(query, read_only = True):
    resp = None
    try:
        db = connect_to_db()
        if read_only:
            resp = pd.read_sql_query(query, db)
        else:
            mycursor = db.cursor()
            mycursor.execute(query)

            db.commit()
        db.close()
    except Exception as e:
        print(e)
    return resp

In [12]:
first_data_season = 2002
first_season = 2008
last_season = 2022

totals_n_last_games = 10
n_last_games = 10
n_last_specific_games = 5

pd.options.mode.chained_assignment = None  # default='warn'

In [13]:
season_games = execute_query(f"SELECT g.id, g.date, g.season, g.is_playoff, g.winner, g.home_id, ht.name as home_name, g.home_pts, g.home_fgm, g.home_fga, g.home_fg_pct, g.home_fg3m, g.home_fg3a, g.home_fg3_pct, g.home_ftm, g.home_fta, g.home_ft_pct, g.home_oreb, g.home_dreb, g.home_reb, g.home_ast, g.home_stl, g.home_blk, g.home_tov, g.home_pf, g.away_id, at.name as away_name, g.away_pts, g.away_fgm, g.away_fga, g.away_fg_pct, g.away_fg3m, g.away_fg3a, g.away_fg3_pct, g.away_ftm, g.away_fta, g.away_ft_pct, g.away_oreb, g.away_dreb, g.away_reb, g.away_ast, g.away_stl, g.away_blk, g.away_tov, g.away_pf, g.home_odds, g.away_odds, g.over_under_line, g.spread_line FROM games AS g LEFT JOIN teams as ht ON g.home_id = ht.id LEFT JOIN teams as at ON g.away_id = at.id WHERE g.season >= {first_data_season} and g.season <= {last_season} ORDER BY g.date ASC")
season_games_plyrs = execute_query(f"SELECT g.id as game_id, g.date, g.season, g.is_playoff, g.winner, g.home_id, g.away_id, pg.team_id, pg.player_id, pg.minutes, pg.pts, pg.fgm, pg.fga, pg.fg_pct, pg.fg3m, pg.fg3a, pg.fg3_pct, pg.ftm, pg.fta, pg.ft_pct, pg.oreb, pg.dreb, pg.reb, pg.ast, pg.stl, pg.blk, pg.tov, pg.pf, pg.plus_minus FROM playergames AS pg LEFT JOIN games as g on pg.game_id = g.id WHERE g.season >= {first_data_season} and g.season <= {last_season} ORDER BY g.date ASC")
teams = execute_query(f"SELECT * FROM teams")

C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [14]:
season_games['home_off_rtg'] = season_games.apply(lambda row: hf.get_team_offensive_rating_game(row, 'H'), axis = 1)
season_games['home_def_rtg'] = season_games.apply(lambda row: hf.get_team_offensive_rating_game(row, 'H'), axis = 1)

season_games['away_off_rtg'] = season_games.apply(lambda row: hf.get_team_offensive_rating_game(row, 'A'), axis = 1)
season_games['away_def_rtg'] = season_games.apply(lambda row: hf.get_team_offensive_rating_game(row, 'A'), axis = 1)

In [15]:
season_games.head()

,id,date,season,is_playoff,winner,home_id,home_name,home_pts,home_fgm,home_fga,...,away_tov,away_pf,home_odds,away_odds,over_under_line,spread_line,home_off_rtg,home_def_rtg,away_off_rtg,away_def_rtg
0,20200001,2002-10-29,2002,0,H,1610612753,Orlando Magic,95,36,81,...,20,22,NaN,NaN,NaN,NaN,98.140496,98.140496,88.176353,88.176353
1,20200002,2002-10-29,2002,0,H,1610612758,Sacramento Kings,94,36,80,...,24,20,NaN,NaN,NaN,NaN,99.787686,99.787686,68.367347,68.367347
2,20200003,2002-10-29,2002,0,A,1610612747,Los Angeles Lakers,82,27,85,...,20,26,NaN,NaN,NaN,NaN,83.673469,83.673469,88.414634,88.414634
3,20200004,2002-10-30,2002,0,A,1610612738,Boston Celtics,96,36,77,...,18,19,NaN,NaN,NaN,NaN,99.792100,99.792100,105.319149,105.319149
4,20200005,2002-10-30,2002,0,H,1610612755,Philadelphia 76ers,95,31,86,...,17,25,NaN,NaN,NaN,NaN,96.153846,96.153846,93.750000,93.750000


In [16]:
season_games_plyrs.head()

,game_id,date,season,is_playoff,winner,home_id,away_id,team_id,player_id,minutes,...,ft_pct,oreb,dreb,reb,ast,stl,blk,tov,pf,plus_minus
0,20200001,2002-10-29,2002,0,H,1610612753,1610612755,1610612755,95,9,...,0.000,0,2,2,0,0,0,1,1,-4
1,20200001,2002-10-29,2002,0,H,1610612753,1610612755,1610612755,243,37,...,0.833,0,6,6,5,4,0,5,4,-1
2,20200001,2002-10-29,2002,0,H,1610612753,1610612755,1610612753,255,33,...,0.889,0,6,6,7,0,0,3,0,6
3,20200001,2002-10-29,2002,0,H,1610612753,1610612755,1610612753,270,14,...,0.000,0,1,1,1,1,0,0,1,2
4,20200001,2002-10-29,2002,0,H,1610612753,1610612755,1610612753,353,34,...,1.000,3,4,7,3,2,0,2,2,8


In [17]:
teams.head()

,id,name,abbreviation
0,1610612737,Atlanta Hawks,ATL
1,1610612738,Boston Celtics,BOS
2,1610612739,Cleveland Cavaliers,CLE
3,1610612740,New Orleans Pelicans,NOP
4,1610612741,Chicago Bulls,CHI


In [18]:
teams_elo_dict = dict()
teams_elo_path = 'teams_elo'

for i, team in teams.iterrows():
    f = open(f"{teams_elo_path}/{team['abbreviation']}.json")
    elo_dict = json.load(f)
    teams_elo_dict[team['id']] = elo_dict

In [19]:
def get_match_info(game_info, stats_team_a, stats_team_b, winner, team_a_pts, team_b_pts):
    return (game_info + stats_team_a + stats_team_b + [winner, team_a_pts, team_b_pts])

def get_team_made_conceded_pct(games, line):
    if not line:
        return None
    
    made = np.array(games['team_pts'])
    conceded = np.array(games['opp_pts'])
    totals = made + conceded
    greater = (totals > line).sum()
    pct = greater*100/len(totals)
    return pct

def get_team_previous_games(season_games, season_games_plyrs, game_date, team_id, opp_id, teams_per, season, line, scenario):    
    response = hf.get_team_previous_games(season_games, team_id, game_date, season)
    if not response: return None
    
    home_previous_games, away_previous_games, previous_games, previous_season_games, home_previous_season_games, away_previous_season_games = response
    
    if len(previous_season_games.index) < 10:
        return None
    
    last_n_games = previous_season_games.iloc[-n_last_games:,:]
    
    # Number of games in the last 4 days
    last_4_days = game_date - pd.Timedelta(days=4)
    number_games_last_4_days = len(last_n_games.loc[last_n_games['date'] >= last_4_days].index)
    
    # Totals info
    totals_overall_pct = get_team_made_conceded_pct(previous_season_games.iloc[-totals_n_last_games:,:], line)
    if scenario == 'H':
        totals_ha_pct = get_team_made_conceded_pct(home_previous_season_games.iloc[-totals_n_last_games:,:], line)
    else:
        totals_ha_pct = get_team_made_conceded_pct(away_previous_season_games.iloc[-totals_n_last_games:,:], line)
    
    # Get last game ELO
    last_game_date = str(previous_season_games.iloc[-1,:]['date'])
    elo = teams_elo_dict[team_id][last_game_date]
    
    # Last n games pct
    pct_last_n_games = hf.get_wl_pct(last_n_games)[0]
    
    # Getting Previous A x B Matchups
    last_matchups = previous_games[previous_games['opp_id'] == opp_id].iloc[-10:,:]
    
    # Getting player information
    teams_per[team_id] = hf.get_team_per_mean(team_id, game_id, game_date, season, season_games_plyrs)
    
    # Season Win Percentage
    season_pct = hf.get_wl_pct(previous_season_games)[0]
    
    # Last n/2 games pct and Season H/A Win Percentage
    if scenario == 'H':
        ha_pct_last_n_games = hf.get_wl_pct(home_previous_season_games.iloc[-n_last_specific_games:,:])[0]
        ha_pct = hf.get_wl_pct(home_previous_season_games)[0]
    else:
        ha_pct_last_n_games = hf.get_wl_pct(away_previous_season_games.iloc[-n_last_specific_games:,:])[0]
        ha_pct = hf.get_wl_pct(away_previous_season_games)[0]
    
    # Matchup Win Percentage
    matchup_pct = hf.get_wl_pct(last_matchups)[0]
    
    # Calculating Current Streak
    streak = hf.current_streak(previous_season_games)
    
    stats_team = hf.get_team_stats (last_n_games, season_pct, teams_per[team_id], elo, matchup_pct, ha_pct, streak, pct_last_n_games, ha_pct_last_n_games, totals_overall_pct, totals_ha_pct)
    
    return stats_team

In [20]:
print("Creating CSV file of all games...")

teams_per = dict()

matches_organized = []
matches_organized_playoffs = []
season = ''

season_games_iterr = season_games.loc[season_games['season'] >= first_season]
season_games_iterr.reset_index(drop=True, inplace=True)

for i, g in season_games_iterr.iterrows():
    clear_output(wait=True)
    print(f"{i}/{len(season_games_iterr.index)}")
    
    season = g['season']
    
    if season < first_season:
        continue

    is_playoffs = g['is_playoff']
        
    game_id = g['id']
    game_date = g['date']

    team_a_id = g['home_id']
    team_b_id = g['away_id']

    team_a_abbv = teams.loc[teams['id'] == team_a_id].iloc[0]['abbreviation']
    team_b_abbv = teams.loc[teams['id'] == team_b_id].iloc[0]['abbreviation']
    
    winner = g['winner']
    
    # Update ELO after stats computed
    team_a_pts = g['home_pts']
    team_b_pts = g['away_pts']

    team_a_odds, team_b_odds = g['home_odds'], g['away_odds']
    
    if not team_a_odds or not team_b_odds:
        continue
    
    stats_team_a = get_team_previous_games(season_games, season_games_plyrs, game_date, team_a_id, team_b_id, teams_per, season, g['over_under_line'], 'H')
    if not stats_team_a:
        continue
    
    stats_team_b = get_team_previous_games(season_games, season_games_plyrs, game_date, team_b_id, team_a_id, teams_per, season, g['over_under_line'], 'A')
    if not stats_team_b:
        continue

    match_info = get_match_info([season, game_date, team_a_abbv, team_b_abbv, team_a_odds, team_b_odds, g['over_under_line'], g['spread_line']], stats_team_a, stats_team_b, winner, team_a_pts, team_b_pts)
    if is_playoffs:
        matches_organized_playoffs.append(match_info)
    else:
        matches_organized.append(match_info)

19142/19143


In [21]:
print("Total matches: {}\nTotal playoffs matches: {}".format(len(matches_organized), len(matches_organized_playoffs)))

Total matches: 15542
Total playoffs matches: 1255


In [22]:
def parse_df_to_csv(dataframe, columns, path, filename):
    if not os.path.exists(path):
        os.makedirs(path)
    final_df = pd.DataFrame(dataframe, columns=columns)
    final_df.to_csv("{}/{}".format(path, filename))
    return final_df

In [23]:
columns = ['SEASON_ID', 'GAME_DATE', 'TEAM_A', 'TEAM_B', 'ODDS_A', 'ODDS_B', 'TOTALS_LINE', 'SPREAD_LINE', 
           'PTS_A', 'PTS_CON_A', 'FG_PCT_A', 'FG3_PCT_A', 'FT_PCT_A', 'REB_A', 'TOV_A', 'SEASON_A_PCT', 'PER_A', 'ELO_A', 'MATCHUP_A_PCT', 'HA_A_PCT', 'STREAK_A', 'LAST_GAMES_PCT_A', 'HA_LAST_GAMES_PCT_A', 'OFF_RTG_A', 'DEF_RTG_A', 'TOTALS_PCT_A', 'TOTALS_HA_PCT_A',
           'PTS_B', 'PTS_CON_B', 'FG_PCT_B', 'FG3_PCT_B', 'FT_PCT_B', 'REB_B', 'TOV_B', 'SEASON_B_PCT', 'PER_B', 'ELO_B', 'MATCHUP_B_PCT', 'HA_B_PCT', 'STREAK_B', 'LAST_GAMES_PCT_B', 'HA_LAST_GAMES_PCT_B', 'OFF_RTG_B', 'DEF_RTG_B', 'TOTALS_PCT_B', 'TOTALS_HA_PCT_B',
           'WINNER', 'GAME_PTS_A', 'GAME_PTS_B']
final_df = parse_df_to_csv(matches_organized, columns, '../data', '{}-{}.csv'.format(first_season, last_season))
parse_df_to_csv(matches_organized_playoffs, columns, '../data/playoffs', '{}-{}.csv'.format(first_season, last_season))

,SEASON_ID,GAME_DATE,TEAM_A,TEAM_B,ODDS_A,ODDS_B,TOTALS_LINE,SPREAD_LINE,PTS_A,PTS_CON_A,...,STREAK_B,LAST_GAMES_PCT_B,HA_LAST_GAMES_PCT_B,OFF_RTG_B,DEF_RTG_B,TOTALS_PCT_B,TOTALS_HA_PCT_B,WINNER,GAME_PTS_A,GAME_PTS_B
0,2008,2009-04-18,CLE,DET,1.100,8.00,176.0,11.5,100.4,92.7,...,-3,0.4,0.2,108.531708,108.531708,90.0,90.0,H,102,84
1,2008,2009-04-18,BOS,CHI,1.213,4.70,198.5,8.5,100.1,97.6,...,-1,0.7,0.6,112.921863,112.921863,70.0,70.0,A,103,105
2,2008,2009-04-18,SAS,DAL,1.526,2.60,187.5,4.0,97.6,95.3,...,-2,0.7,0.4,114.346804,114.346804,80.0,60.0,A,97,105
3,2008,2009-04-18,POR,HOU,1.444,2.85,182.0,5.5,100.7,84.9,...,-1,0.6,0.4,108.330667,108.330667,60.0,50.0,A,81,108
4,2008,2009-04-19,ORL,PHI,1.174,5.63,192.0,9.5,93.0,91.3,...,-1,0.4,0.2,108.802288,108.802288,70.0,90.0,A,98,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,2022,2023-06-01,DEN,MIA,NaN,NaN,NaN,NaN,117.5,109.4,...,-1,0.6,0.6,112.661927,112.661927,0.0,0.0,H,104,93
1251,2022,2023-06-04,DEN,MIA,NaN,NaN,NaN,NaN,115.4,108.0,...,-1,0.5,0.6,110.956170,110.956170,0.0,0.0,A,108,111
1252,2022,2023-06-07,MIA,DEN,NaN,NaN,NaN,NaN,106.4,104.1,...,-1,0.7,0.6,121.162081,121.162081,0.0,0.0,A,94,109
1253,2022,2023-06-09,MIA,DEN,NaN,NaN,NaN,NaN,106.2,105.8,...,-1,0.8,0.8,121.902907,121.902907,0.0,0.0,A,95,108
